In [1]:
import pandas as pd
import numpy as np
import math

### Carga dos dados

In [3]:
# definir arquivo Pickle de entrada
#arquivo = 'dados/afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'dados/cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [4]:
df = pd.read_pickle(arquivo)
df.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,15925,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,11497,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,7515,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,7370,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [5]:
df.shape

(41146, 18)

### Expansão dos dados agregados

In [7]:
%%time
df2 = df.head(50000).copy()
#df2 = df.copy()
menor = df2['QTDE'].min()
maior = df2['QTDE'].max()
print(df2.shape[0], menor, maior, math.ceil(np.sqrt(maior / menor)))
#df2['TAM'] = df2['QTDE'].apply(lambda x: math.ceil(np.sqrt(x / menor)))
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(
    lambda x: [x[0]] * math.ceil(np.sqrt(x[1] / menor)), axis=1)
dfe = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()
dfe.head(1000)

41146 1 20389 143
CPU times: user 404 ms, sys: 5.62 ms, total: 410 ms
Wall time: 406 ms


,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0


In [8]:
# usar os dados expandidos invés dos agregados
dfo = df # guardar o dataframe original
df = dfe # apontar para o dataframe expandido

### Uso dos dados originais

In [9]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (82709, 16) (82709,)


In [10]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (55415, 16) (27294, 16) (55415,) (27294,)


In [13]:
X_train.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
THIALLAN,14,1,12,12,1,9,8,20,0,0,0,0,0,0,0,0
FLAVIANE,5,14,1,9,22,1,12,6,0,0,0,0,0,0,0,0
CLAUDERINO,15,14,9,18,5,4,21,1,12,3,0,0,0,0,0,0
EDUARDO,15,4,18,1,21,4,5,0,0,0,0,0,0,0,0,0
ALEXANDRO,15,18,4,14,1,24,5,12,1,0,0,0,0,0,0,0


In [14]:
y_train.head()

NOME
THIALLAN      1
FLAVIANE      0
CLAUDERINO    1
EDUARDO       1
ALEXANDRO     1
Name: SEXO, dtype: uint8

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [21]:
# definição do modelo de Deep Learning
modelo = Sequential()
modelo.add(Dense(10, input_dim = 16, activation = 'relu')) # 4: núm. atributos
modelo.add(Dense(10, activation = 'relu'))
modelo.add(Dense(2, activation = 'softmax')) # 3: núm. classes

# compilação do modelo
modelo.compile(loss = 'binary_crossentropy',
               optimizer = 'sgd',
               metrics = ['accuracy'])

modelo

In [22]:
# treinamento do modelo
modelo.fit(X_train, y_train, epochs = 300, batch_size = 10)
modelo

Epoch 1/300


ValueError: in user code:

    /dados/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /dados/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /dados/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /dados/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /dados/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /dados/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:838 ru

In [ ]:
# Avalia o modelo
scores = modelo.evaluate(X_teste, y_teste)
print("\nAcurácia: %.2f%%" % (scores[1]*100))

https://machinelearningmastery.com/tensorflow-tutorial-deep-learning-with-tf-keras/

In [18]:
import tensorflow as tf

In [20]:
print(tf.__version__)

2.5.0


In [23]:
# example of a model defined with the sequential api
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# define the model
model = Sequential()
model.add(Dense(10, input_shape=(8,)))
model.add(Dense(1))
model

In [25]:
# example of a model defined with the sequential api
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# define the model
model = Sequential()
model.add(Dense(100, input_shape=(8,)))
model.add(Dense(80))
model.add(Dense(30))
model.add(Dense(10))
model.add(Dense(5))
model.add(Dense(1))
model

In [26]:
# example of a model defined with the functional api
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense

# define the layers
x_in = Input(shape=(8,))
x = Dense(10)(x_in)
x_out = Dense(1)(x)

# define the model
model = Model(inputs=x_in, outputs=x_out)
model

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1125)>